# Neural Network Example

Build a 2-hidden layers fully connected neural network (a.k.a multilayer perceptron) and a 2 layer-CNN with TensorFlow.

- Author: Zheng Liu

## Neural Network Overview

<img src="http://cs231n.github.io/assets/nn1/neural_net2.jpeg" alt="nn" style="width: 400px;"/>

## MNIST Dataset Overview

This example is using MNIST handwritten digits. The dataset contains 60,000 examples for training and 10,000 examples for testing. The digits have been size-normalized and centered in a fixed-size image (28x28 pixels) with values from 0 to 1. For simplicity, each image has been flattened and converted to a 1-D numpy array of 784 features (28*28).

![MNIST Dataset](http://neuralnetworksanddeeplearning.com/images/mnist_100_digits.png)

More info: http://yann.lecun.com/exdb/mnist/

In [ ]:
from __future__ import print_function

# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

import tensorflow as tf

In [ ]:
# Parameters
num_steps = 500
batch_size = 128
display_step = 100

# Network Parameters
n_hidden_1 = 256 # 1st layer number of neurons
n_hidden_2 = 64 # 2nd layer number of neurons
num_input = 784 # MNIST data input (img shape: 28*28)
num_classes = 10 # MNIST total classes (0-9 digits)

# tf Graph input
X = tf.placeholder("float", [None, num_input])
Y = tf.placeholder("float", [None, num_classes])

In [ ]:
# Create model
def neural_net_raw(x):
    # layers weight & bias
    w1 = tf.Variable(tf.random_normal([num_input, n_hidden_1]))
    w2 = tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2]))
    w_out = tf.Variable(tf.random_normal([n_hidden_2, num_classes]))
    b1 = tf.Variable(tf.random_normal([n_hidden_1]))
    b2 = tf.Variable(tf.random_normal([n_hidden_2]))
    b_out = tf.Variable(tf.random_normal([num_classes]))
    
    # start to build the network (note: this network doesn't have activation layers)
    ly1 = tf.add(tf.matmul(x, w1), b1)
    ly2 = tf.add(tf.matmul(ly1, w2), b2)
    out_layer = tf.matmul(ly2, w_out) + b_out
    return out_layer

In [ ]:
def neural_net_easy(x):
    # start to build the network (note: this network doesn't have activation layers)
    ly1 = tf.layers.dense(x, n_hidden_1)
    ly2 = tf.layers.dense(ly1, n_hidden_2)
    out_layer = tf.layers.dense(ly2, num_classes)
    return out_layer

In [ ]:
def neural_net_cnn(x):
    # MNIST data input is a 1-D vector of 784 features (28*28 pixels)
    # Reshape to match picture format [Height x Width x Channel]
    # Tensor input become 4-D: [Batch Size, Height, Width, Channel]
    x = tf.reshape(x, shape=[-1, 28, 28, 1])
    
    # first layer
    conv1 = tf.layers.conv2d(inputs = x, filters = 16, kernel_size = 2, strides = 1, activation = tf.nn.relu)
    pool1 = tf.layers.max_pooling2d(inputs = conv1, pool_size = 2, strides = 2)
    # second layer
    conv2 = tf.layers.conv2d(inputs = pool1, filters = 16, kernel_size = 2, strides = 1, activation = tf.nn.relu)
    pool2 = tf.layers.max_pooling2d(inputs = conv2, pool_size = 2, strides = 2)
    # output layer
    fc1 = tf.contrib.layers.flatten(pool2)
    out_layer = tf.layers.dense(fc1, num_classes)
    
    return out_layer

In [ ]:
# Construct model
#prediction = neural_net_raw(X)
#prediction = neural_net_easy(X)
prediction = neural_net_cnn(X)

# Define loss and optimizer
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=prediction, labels=Y))
optimizer = tf.train.AdamOptimizer()
train = optimizer.minimize(loss)

# Evaluate model (uses prediction accuracy)
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [ ]:
# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    for step in range(num_steps):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Run optimization op (backprop)
        sess.run(train, feed_dict={X: batch_x, Y: batch_y})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            current_loss, acc = sess.run([loss, accuracy], feed_dict={X: batch_x,
                                                                 Y: batch_y})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(current_loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))

    print("Optimization Finished!")

    # Calculate accuracy for MNIST test images
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={X: mnist.test.images,
                                      Y: mnist.test.labels}))

In [ ]:
total_parameters = 0
#iterating over all variables
for variable in tf.trainable_variables():  
    local_parameters=1
    shape = variable.get_shape()  #getting shape of a variable
    for i in shape:
        local_parameters*=i.value  #mutiplying dimension values
    total_parameters+=local_parameters
print(total_parameters) 

# Training results

neural_net_raw:
- testing accuracy: 0.8426
- number of parameters: 218.06k

neural_net_easy:
- testing accuracy: 0.0.9154
- number of parameters: 218.06k

neural_net_cnn:
- testing accuracy:0.9516
- number of parameters: 6.89k
    

